<a href="https://colab.research.google.com/github/rdthemis/ADTD/blob/main/Trabalho%20Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trabalho final de Análise de Dados e Tomada de Decisões 2023
Desafio
O naufrágio do RMS Titanic
Após colidir com um iceberg em 15 de abril de 1912 o návio
resultando na morte de 1502 dos 2224 passageiros e tripulantes.

1° etapa
Análise Exploratória e Visualização de Dados

importação das bibliotecas

In [24]:
import pandas as pd
from pandas.io.formats.format import math
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import r2_score, accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

importação dos dados para dataset com as informações do RMS Titanic

In [2]:
titanic = pd.read_csv('/content/drive/MyDrive/train.csv')

Exibindo informações sobre os dados

In [3]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Realizando leitura do dataset por padrão será exibido 5 linhas mas podemos informar qualquer quantidade.

In [30]:
titanic.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,Não Cad,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,Não Cad,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,Não Cad,S
5,6,0,3,"Moran, Mr. James",male,29.0,0,0,330877,8.4583,Não Cad,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,Não Cad,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,Não Cad,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,Não Cad,C


Verifica-se que existe id's de passageiros sem informação de qual cabine ele estava, iremos verificar se existe algum NUll também.

In [20]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Vamos tratar essas informações, excluir não é uma opção pois perderiamos detalhes inportantee spara nossa análise futuramente. Então vamos preencher esses campos com alguma informação, sendo de acordo com o contexto para não mascarar os dados.

Primeiro 177 id's com idade sem informação, neste caso uma solução séria pegar a média das idades de todos os passageiros e substituir os nulos.

In [21]:
idadeMedia = titanic['Age'].mean()
idadeMedia

29.69911764705882

substituindo a idade

In [25]:
titanic.update(titanic['Age'].fillna((math.floor(idadeMedia))))

Foi utilizado o math da biblioteca Pandas para arredondar a idade.
Iremos ferificar se a alterar foi feita com sucesso.

In [26]:
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Iremos tratar as cabines sem informação, uma solução simples que ao olhar identificamos qual a informação contida é substituir NaN por Cabine Não Cadastrada.

In [28]:
titanic.update(titanic['Cabin'].fillna('Não Cad'))
titanic.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       2
dtype: int64

In [42]:
emb = titanic.loc[titanic['Embarked'].isnull()]
emb['Embarked']

61     NaN
829    NaN
Name: Embarked, dtype: object

Iremos tratar está informação de Embarked comNão Informado

In [43]:
titanic.update(titanic['Embarked'].fillna(('Não Inf')))
titanic.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

Descrição das características dos tripulantes do titanic
Idade

In [46]:
titanic['Age'].describe()

count    891.000000
mean      29.560236
std       13.005010
min        0.420000
25%       22.000000
50%       29.000000
75%       35.000000
max       80.000000
Name: Age, dtype: float64

Sexo

In [81]:
for col_name in titanic.columns:
    if titanic[col_name].dtypes == 'object':
        unique_cat = len(titanic[col_name].unique())
        print("Feature '{col_name}' has {unique_cat} unique   categories".format(col_name=col_name, unique_cat=unique_cat))

Feature 'Name' has 891 unique   categories
Feature 'Sex' has 2 unique   categories
Feature 'Ticket' has 681 unique   categories
Feature 'Cabin' has 148 unique   categories
Feature 'Embarked' has 4 unique   categories
